## Option 1: Use ERDDAP DAP endpoint to obtain an OPeNDAP 'Series' output

Transform and reduce via xarray the DAP Series response to a multi-dimensional netCDF file

In [1]:
import xarray as xr
import numpy as np


ds = xr.open_dataset("http://erddap.sensors.ioos.us/erddap/tabledap/gov_usgs_cmgp_buzz_bay_265")
ds

<xarray.Dataset>
Dimensions:                                                  (s: 27617)
Dimensions without coordinates: s
Data variables:
    s.time                                                   (s) datetime64[ns] ...
    s.latitude                                               (s) float64 ...
    s.longitude                                              (s) float64 ...
    s.z                                                      (s) float64 ...
    s.backscatter_intensity_2651_a                           (s) float64 ...
    s.sea_water_velocity_to_direction_2651ds_a               (s) float64 ...
    s.sea_water_speed_2651ds_a                               (s) float64 ...
    s.eastward_sea_water_velocity_2651ds_a                   (s) float64 ...
    s.northward_sea_water_velocity_2651ds_a                  (s) float64 ...
    s.sea_water_pressure_cm_time__standard_deviation_2651_a  (s) float64 ...
    s.sea_water_pressure_2651_a                              (s) float64 ...
    s.sea_water_temperature_2651_a                           (s) float64 ...
    s.station                                                (s) |S64 ...
Attributes:
    cdm_altitude_proxy:            z
    cdm_data_type:                 TimeSeriesProfile
    cdm_profile_variables:         time
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             None,feedback@axiomdatascience.com
    contributor_name:              Mid-Atlantic Coastal Ocean Observing Syste...
    contributor_role:              funder,processor
    contributor_role_vocabulary:   NERC
    contributor_url:               https://maracoos.org/,https://www.axiomdat...
    Conventions:                   IOOS-1.2, CF-1.6, ACDD-1.3, NCCSV-1.0
    creator_country:               USA
    creator_institution:           USGS Coastal and Marine Geology Program (U...
    creator_name:                  USGS Coastal and Marine Geology Program (U...
    creator_sector:                gov_federal
    creator_type:                  institution
    creator_url:                   http://marine.usgs.gov/
    Easternmost_Easting:           -70.68755340576172
    featureType:                   TimeSeriesProfile
    geospatial_lat_max:            41.6508903503418
    geospatial_lat_min:            41.6508903503418
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -70.68755340576172
    geospatial_lon_min:            -70.68755340576172
    geospatial_lon_units:          degrees_east
    geospatial_vertical_max:       13.0
    geospatial_vertical_min:       0.0
    geospatial_vertical_positive:  up
    geospatial_vertical_units:     m
    history:                       Downloaded from USGS Coastal and Marine Ge...
    id:                            56575
    infoUrl:                       https://sensors.ioos.us/#metadata/56575/st...
    institution:                   USGS Coastal and Marine Geology Program (U...
    license:                       The data may be used and redistributed for...
    naming_authority:              com.axiomdatascience
    Northernmost_Northing:         41.6508903503418
    platform:                      fixed
    platform_name:                 BUZZ_BAY - 265
    platform_vocabulary:           http://mmisw.org/ont/ioos/platform
    processing_level:              Level 2
    publisher_country:             USA
    publisher_institution:         USGS Coastal and Marine Geology Program (U...
    publisher_name:                USGS Coastal and Marine Geology Program (U...
    publisher_sector:              gov_federal
    publisher_type:                institution
    publisher_url:                 http://marine.usgs.gov/
    references:                    https://stellwagen.er.usgs.gov/buzz_bay.ht...
    sourceUrl:                     https://sensors.axds.co/api/
    Southernmost_Northing:         41.6508903503418
    standard_name_vocabulary:      CF Standard Name Table v72
    summary:                       Timese

`s` is the dimension and all variables are appended an `s.`.

In [3]:
ds["s.backscatter_intensity_2651_a"]

<xarray.DataArray 's.backscatter_intensity_2651_a' (s: 27617)>
array([2.09599, 2.09599, 2.14481, ..., 0.31406, 0.30918, 0.29453])
Dimensions without coordinates: s
Attributes:
    discriminant:       2651_a
    id:                 423972
    ioos_category:      Other
    long_name:          Backscatter Intensity
    platform:           station
    standard_name:      backscatter_intensity
    standard_name_url:  http://mmisw.org/ont/unknown/parameter/backscatter_in...
    units:              1

### We need to split the coordinates and variables, rename, re-attach coords/dims, and re-create the dataset.

In [4]:
def fix_erddap(ds, dimension, coordinates):
    """Does not handle more than 1 dimension!"""
    new_vars, coords = {}, {}
    for variable in ds.data_vars:
        new_name = variable.split(".")[-1]
        if new_name in coordinates + [dimension]:
            if new_name not in dimension:
                # avoid repeating values due to erddap filling the time dimension
                #da = ds[variable][0:1]
                da = xr.DataArray(np.unique(ds[variable]))
            else:
                da = ds[variable]
            new_var = da.rename(new_name)
            new_var = xr.IndexVariable(new_name, new_var)
            coords.update({new_name: new_var})
        elif new_name not in coordinates + [dimension]:
            new_var = ds[variable].rename(new_name).rename({"s": dimension})
            new_vars.update({new_name: new_var})

    return xr.Dataset(new_vars, coords=coords, attrs=ds.attrs)

In [5]:
dimension = "time"
coordinates = ["station","latitude", "longitude", "z"]

new_ds_dap_series = fix_erddap(ds, dimension, coordinates)
new_ds_dap_series

<xarray.Dataset>
Dimensions:                                                (latitude: 1, longitude: 1, station: 1, time: 27617, z: 1)
Coordinates:
  * time                                                   (time) datetime64[ns] 1982-10-31T14:31:52 ... 1982-08-20T16:31:52
  * latitude                                               (latitude) float64 41.65
  * longitude                                              (longitude) float64 -70.69
  * z                                                      (z) float64 -13.0
  * station                                                (station) |S64 b'BUZZ_BAY - 265'
Data variables:
    backscatter_intensity_2651_a                           (time) float64 2.096 ... 0.2945
    sea_water_velocity_to_direction_2651ds_a               (time) float64 nan ... 207.3
    sea_water_speed_2651ds_a                               (time) float64 nan ... 0.2041
    eastward_sea_water_velocity_2651ds_a                   (time) float64 nan ... -0.09353
    northward_sea_water_velocity_2651ds_a                  (time) float64 nan ... -0.1814
    sea_water_pressure_cm_time__standard_deviation_2651_a  (time) float64 0.00885 ... 0.03115
    sea_water_pressure_2651_a                              (time) float64 21.03 ... 20.9
    sea_water_temperature_2651_a                           (time) float64 12.72 ... 21.95
Attributes:
    cdm_altitude_proxy:            z
    cdm_data_type:                 TimeSeriesProfile
    cdm_profile_variables:         time
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             None,feedback@axiomdatascience.com
    contributor_name:              Mid-Atlantic Coastal Ocean Observing Syste...
    contributor_role:              funder,processor
    contributor_role_vocabulary:   NERC
    contributor_url:               https://maracoos.org/,https://www.axiomdat...
    Conventions:                   IOOS-1.2, CF-1.6, ACDD-1.3, NCCSV-1.0
    creator_country:               USA
    creator_institution:           USGS Coastal and Marine Geology Program (U...
    creator_name:                  USGS Coastal and Marine Geology Program (U...
    creator_sector:                gov_federal
    creator_type:                  institution
    creator_url:                   http://marine.usgs.gov/
    Easternmost_Easting:           -70.68755340576172
    featureType:                   TimeSeriesProfile
    geospatial_lat_max:            41.6508903503418
    geospatial_lat_min:            41.6508903503418
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -70.68755340576172
    geospatial_lon_min:            -70.68755340576172
    geospatial_lon_units:          degrees_east
    geospatial_vertical_max:       13.0
    geospatial_vertical_min:       0.0
    geospatial_vertical_positive:  up
    geospatial_vertical_units:     m
    history:                       Downloaded from USGS Coastal and Marine Ge...
    id:                            56575
    infoUrl:                       https://sensors.ioos.us/#metadata/56575/st...
    institution:                   USGS Coastal and Marine Geology Program (U...
    license:                       The data may be used and redistributed for...
    naming_authority:              com.axiomdatascience
    Northernmost_Northing:         41.6508903503418
    platform:                      fixed
    platform_name:                 BUZZ_BAY - 265
    platform_vocabulary:           http://mmisw.org/ont/ioos/platform
    processing_level:              Level 2
    publisher_country:             USA
    publisher_institution:         USGS Coastal and Marine Geology Program (U...
    publisher_name:                USGS Coastal and Marine Geology Program (U...
    publisher_sector:              gov_federal
    publisher_type:                institution
    publisher_url:                 http://marine.usgs.gov/
    references:                    https://stellwagen.er.usgs.gov/buzz_bay.ht...
    sourceUrl

Write out the resulting reduced file to disk to compare size:

In [7]:
%%time
# write to single netcdf
filename = "output/gov_usgs_cmgp_buzz_bay_265_series_out.nc"
new_ds_dap_series.to_netcdf(filename)

CPU times: user 29.2 ms, sys: 5.45 ms, total: 34.7 ms
Wall time: 33.6 ms


## Option 2: Use ERDDAP .nc endpoint to obtain the flattened netCDF (.cf) representation

Then work directly with the downloaded netCDF file to perform the same transformations.  The netCDF file response requires us to download all the data but the workaround is simpler?

In [8]:
url = "http://erddap.sensors.ioos.us/erddap/tabledap/gov_usgs_cmgp_buzz_bay_265.nc"

!wget $url

--2020-06-16 10:42:01--  http://erddap.sensors.ioos.us/erddap/tabledap/gov_usgs_cmgp_buzz_bay_265.nc
Resolving erddap.sensors.ioos.us (erddap.sensors.ioos.us)... 216.218.248.42
Connecting to erddap.sensors.ioos.us (erddap.sensors.ioos.us)|216.218.248.42|:80... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/x-netcdf]
Saving to: ‘gov_usgs_cmgp_buzz_bay_265.nc’

gov_usgs_cmgp_buzz_     [         <=>        ]   2.91M   362KB/s    in 18s     

2020-06-16 10:42:21 (170 KB/s) - ‘gov_usgs_cmgp_buzz_bay_265.nc’ saved [3047016]



In [9]:
ds = xr.open_dataset("gov_usgs_cmgp_buzz_bay_265.nc")
ds

<xarray.Dataset>
Dimensions:                                                (row: 27617)
Dimensions without coordinates: row
Data variables:
    time                                                   (row) datetime64[ns] ...
    latitude                                               (row) float64 ...
    longitude                                              (row) float64 ...
    z                                                      (row) float64 ...
    backscatter_intensity_2651_a                           (row) float64 ...
    sea_water_velocity_to_direction_2651ds_a               (row) float64 ...
    sea_water_speed_2651ds_a                               (row) float64 ...
    eastward_sea_water_velocity_2651ds_a                   (row) float64 ...
    northward_sea_water_velocity_2651ds_a                  (row) float64 ...
    sea_water_pressure_cm_time__standard_deviation_2651_a  (row) float64 ...
    sea_water_pressure_2651_a                              (row) float64 ...
    sea_water_temperature_2651_a                           (row) float64 ...
    station                                                (row) object ...
Attributes:
    cdm_altitude_proxy:            z
    cdm_data_type:                 TimeSeriesProfile
    cdm_profile_variables:         time
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             None,feedback@axiomdatascience.com
    contributor_name:              Mid-Atlantic Coastal Ocean Observing Syste...
    contributor_role:              funder,processor
    contributor_role_vocabulary:   NERC
    contributor_url:               https://maracoos.org/,https://www.axiomdat...
    Conventions:                   IOOS-1.2, CF-1.6, ACDD-1.3, NCCSV-1.0
    creator_country:               USA
    creator_institution:           USGS Coastal and Marine Geology Program (U...
    creator_name:                  USGS Coastal and Marine Geology Program (U...
    creator_sector:                gov_federal
    creator_type:                  institution
    creator_url:                   http://marine.usgs.gov/
    Easternmost_Easting:           -70.68755340576172
    featureType:                   TimeSeriesProfile
    geospatial_lat_max:            41.6508903503418
    geospatial_lat_min:            41.6508903503418
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -70.68755340576172
    geospatial_lon_min:            -70.68755340576172
    geospatial_lon_units:          degrees_east
    geospatial_vertical_positive:  up
    geospatial_vertical_units:     m
    history:                       Downloaded from USGS Coastal and Marine Ge...
    id:                            56575
    infoUrl:                       https://sensors.ioos.us/#metadata/56575/st...
    institution:                   USGS Coastal and Marine Geology Program (U...
    license:                       The data may be used and redistributed for...
    naming_authority:              com.axiomdatascience
    Northernmost_Northing:         41.6508903503418
    platform:                      fixed
    platform_name:                 BUZZ_BAY - 265
    platform_vocabulary:           http://mmisw.org/ont/ioos/platform
    processing_level:              Level 2
    publisher_country:             USA
    publisher_institution:         USGS Coastal and Marine Geology Program (U...
    publisher_name:                USGS Coastal and Marine Geology Program (U...
    publisher_sector:              gov_federal
    publisher_type:                institution
    publisher_url:                 http://marine.usgs.gov/
    references:                    https://stellwagen.er.usgs.gov/buzz_bay.ht...
    sourceUrl:                     https://sensors.axds.co/api/
    Southernmost_Northing:         41.6508903503418
    standard_name_vocabulary:      CF Standard Name Table v72
    summary:                       Timeseries data from 'BUZZ_BAY - 265' (urn...
    time_coverage_end:             

Here we reduce the dimensions of data variables that are expected to be non-varying within the timeseries (latitude, longitude, z, station) - should also match those specified in ERDDAP's 'cdm_timeseries_variables' attribute.  

Numpy unique() can be used to determine distinct coordinate values.


In [10]:
coords={}
for dim in ['latitude','longitude','z','station']:
    da = xr.IndexVariable(dim, np.unique(ds[dim]))
    coords.update({dim: da})
new_ds_wget = ds.assign_coords(coords)  
new_ds_wget


<xarray.Dataset>
Dimensions:                                                (latitude: 1, longitude: 1, row: 27617, station: 1, z: 1)
Coordinates:
  * latitude                                               (latitude) float64 41.65
  * longitude                                              (longitude) float64 -70.69
  * z                                                      (z) float64 -13.0
  * station                                                (station) object 'BUZZ_BAY - 265'
Dimensions without coordinates: row
Data variables:
    time                                                   (row) datetime64[ns] 1982-10-31T14:31:52 ... 1982-08-20T16:31:52
    backscatter_intensity_2651_a                           (row) float64 2.096 ... 0.2945
    sea_water_velocity_to_direction_2651ds_a               (row) float64 nan ... 207.3
    sea_water_speed_2651ds_a                               (row) float64 nan ... 0.2041
    eastward_sea_water_velocity_2651ds_a                   (row) float64 nan ... -0.09353
    northward_sea_water_velocity_2651ds_a                  (row) float64 nan ... -0.1814
    sea_water_pressure_cm_time__standard_deviation_2651_a  (row) float64 0.00885 ... 0.03115
    sea_water_pressure_2651_a                              (row) float64 21.03 ... 20.9
    sea_water_temperature_2651_a                           (row) float64 12.72 ... 21.95
Attributes:
    cdm_altitude_proxy:            z
    cdm_data_type:                 TimeSeriesProfile
    cdm_profile_variables:         time
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             None,feedback@axiomdatascience.com
    contributor_name:              Mid-Atlantic Coastal Ocean Observing Syste...
    contributor_role:              funder,processor
    contributor_role_vocabulary:   NERC
    contributor_url:               https://maracoos.org/,https://www.axiomdat...
    Conventions:                   IOOS-1.2, CF-1.6, ACDD-1.3, NCCSV-1.0
    creator_country:               USA
    creator_institution:           USGS Coastal and Marine Geology Program (U...
    creator_name:                  USGS Coastal and Marine Geology Program (U...
    creator_sector:                gov_federal
    creator_type:                  institution
    creator_url:                   http://marine.usgs.gov/
    Easternmost_Easting:           -70.68755340576172
    featureType:                   TimeSeriesProfile
    geospatial_lat_max:            41.6508903503418
    geospatial_lat_min:            41.6508903503418
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -70.68755340576172
    geospatial_lon_min:            -70.68755340576172
    geospatial_lon_units:          degrees_east
    geospatial_vertical_positive:  up
    geospatial_vertical_units:     m
    history:                       Downloaded from USGS Coastal and Marine Ge...
    id:                            56575
    infoUrl:                       https://sensors.ioos.us/#metadata/56575/st...
    institution:                   USGS Coastal and Marine Geology Program (U...
    license:                       The data may be used and redistributed for...
    naming_authority:              com.axiomdatascience
    Northernmost_Northing:         41.6508903503418
    platform:                      fixed
    platform_name:                 BUZZ_BAY - 265
    platform_vocabulary:           http://mmisw.org/ont/ioos/platform
    processing_level:              Level 2
    publisher_country:             USA
    publisher_institution:         USGS Coastal and Marine Geology Program (U...
    publisher_name:                USGS Coastal and Marine Geology Program (U...
    publisher_sector:              gov_federal
    publisher_type:                institution
    publisher_url:                 http://marine.usgs.gov/
    references:                    https://stellwagen.er.usgs.gov/buzz_bay.ht...
    sourceUrl:                     https://sensors.axds.co/api/
 

Swap the default 'row' dimsension with 'time'

In [11]:
new_ds_wget = new_ds_wget.swap_dims({"row": "time"})
new_ds_wget

<xarray.Dataset>
Dimensions:                                                (latitude: 1, longitude: 1, station: 1, time: 27617, z: 1)
Coordinates:
  * time                                                   (time) datetime64[ns] 1982-10-31T14:31:52 ... 1982-08-20T16:31:52
  * latitude                                               (latitude) float64 41.65
  * longitude                                              (longitude) float64 -70.69
  * z                                                      (z) float64 -13.0
  * station                                                (station) object 'BUZZ_BAY - 265'
Data variables:
    backscatter_intensity_2651_a                           (time) float64 2.096 ... 0.2945
    sea_water_velocity_to_direction_2651ds_a               (time) float64 nan ... 207.3
    sea_water_speed_2651ds_a                               (time) float64 nan ... 0.2041
    eastward_sea_water_velocity_2651ds_a                   (time) float64 nan ... -0.09353
    northward_sea_water_velocity_2651ds_a                  (time) float64 nan ... -0.1814
    sea_water_pressure_cm_time__standard_deviation_2651_a  (time) float64 0.00885 ... 0.03115
    sea_water_pressure_2651_a                              (time) float64 21.03 ... 20.9
    sea_water_temperature_2651_a                           (time) float64 12.72 ... 21.95
Attributes:
    cdm_altitude_proxy:            z
    cdm_data_type:                 TimeSeriesProfile
    cdm_profile_variables:         time
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             None,feedback@axiomdatascience.com
    contributor_name:              Mid-Atlantic Coastal Ocean Observing Syste...
    contributor_role:              funder,processor
    contributor_role_vocabulary:   NERC
    contributor_url:               https://maracoos.org/,https://www.axiomdat...
    Conventions:                   IOOS-1.2, CF-1.6, ACDD-1.3, NCCSV-1.0
    creator_country:               USA
    creator_institution:           USGS Coastal and Marine Geology Program (U...
    creator_name:                  USGS Coastal and Marine Geology Program (U...
    creator_sector:                gov_federal
    creator_type:                  institution
    creator_url:                   http://marine.usgs.gov/
    Easternmost_Easting:           -70.68755340576172
    featureType:                   TimeSeriesProfile
    geospatial_lat_max:            41.6508903503418
    geospatial_lat_min:            41.6508903503418
    geospatial_lat_units:          degrees_north
    geospatial_lon_max:            -70.68755340576172
    geospatial_lon_min:            -70.68755340576172
    geospatial_lon_units:          degrees_east
    geospatial_vertical_positive:  up
    geospatial_vertical_units:     m
    history:                       Downloaded from USGS Coastal and Marine Ge...
    id:                            56575
    infoUrl:                       https://sensors.ioos.us/#metadata/56575/st...
    institution:                   USGS Coastal and Marine Geology Program (U...
    license:                       The data may be used and redistributed for...
    naming_authority:              com.axiomdatascience
    Northernmost_Northing:         41.6508903503418
    platform:                      fixed
    platform_name:                 BUZZ_BAY - 265
    platform_vocabulary:           http://mmisw.org/ont/ioos/platform
    processing_level:              Level 2
    publisher_country:             USA
    publisher_institution:         USGS Coastal and Marine Geology Program (U...
    publisher_name:                USGS Coastal and Marine Geology Program (U...
    publisher_sector:              gov_federal
    publisher_type:                institution
    publisher_url:                 http://marine.usgs.gov/
    references:                    https://stellwagen.er.usgs.gov/buzz_bay.ht...
    sourceUrl:                     https://sensors.axds.co/api/
    Southernmost_Northing: 

Write out the resulting reduced file to disk to compare size:

In [12]:
%%time
# write to single netcdf
filename = "output/gov_usgs_cmgp_buzz_bay_265_nc_out.nc"
new_ds_wget.to_netcdf(filename)

CPU times: user 38.8 ms, sys: 0 ns, total: 38.8 ms
Wall time: 36.5 ms
